# Projeto 3: "Efeitos de vídeo 

## 1: Quais transformações foram implementadas? Explique como cada matriz de transformação afeta os pontos da imagem. 

**No código foram implementadas as seguintes transformações:**

#### 1. Rotação ($R$):

$$
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) & 0 \\
\sin(\theta) & \cos(\theta) & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

Onde:
- θ é o ângulo de rotação em radianos.

Com essa matriz, podemos rotacionar os pontos da imagem em volta da origem (0, 0), ou seja, o centro da imagem. Vale ressaltar que precisamos que o centro da imagem esteja na origem pois o ponto que os pixeis se rotacionam por padrão é a origem, (0, 0).

Ao multiplicarmos os pontos da imagem pela matriz de rotação, teríamos: 

$$(x', y') = (x \cdot \cos(\theta) - y \cdot \sin(\theta), x \cdot \sin(\theta) + y \cdot \cos(\theta))$$

- onde $x$ e $y$ são as coordenadas do pixel na imagem original.

#### 2. Escalonamento (Mudança de Escala) ($S$):

$$
\begin{bmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

Onde:
- $s_x$ é o fator de escala horizontal.
- $s_y$ é o fator de escala vertical.

Esta matriz é responsável pelo aumento ou redução do tamanho dos pontos da imagem.

Quando multiplicamos o vetor de coordenadas de um pixel pela matriz de escala, o resultado é um vetor de coordenadas que é maior ou menor que o original, ou seja:

Se originalmente tinhamos um pixel com coordenadas de ($x$, $y$), após a multiplicação pela matriz de escala, teremos um pixel com coordenadas de ($s_x \cdot x$, $s_y \cdot y$). Simulando assim, o zoom in ou zoom out.


#### 3. Translações ($T$ e $T^{-1}$):

Translação($T$)
$$
\begin{bmatrix}
1 & 0 & \frac{\text{largura da imagem}}{2} \\
0 & 1 & \frac{\text{altura da imagem}}{2} \\
0 & 0 & 1
\end{bmatrix}
$$

Translação inversa($T^{-1}$)
$$
\begin{bmatrix}
1 & 0 & -\frac{\text{largura da imagem}}{2} \\
0 & 1 & -\frac{\text{altura da imagem}}{2} \\
0 & 0 & 1
\end{bmatrix}
$$

Onde:
- A Matriz $T$ translada a imagem para o centro.
- A Matriz $T^{-1}$ translada a imagem de volta para a posição original.

Precisamos de duas matrizes de translação para que possamos transladar os pontos da imagem para o centro da imagem, aplicar a transformação desejada e depois transladar os pontos de volta para a posição original.



### Como as Matrizes de Transformação afetam os pontos da Imagem:

A transformação final aplicada a cada ponto $X$ da imagem é calculada como:

$$
X_{final} = T \cdot R \cdot S \cdot T^{-1} \cdot X
$$

Entretanto, para evitar "buracos" na imagem resultante (onde alguns pixels não são preenchidos), a transformação inversa é aplicada para mapear cada ponto da imagem de saída para um ponto correspondente na imagem de entrada.

Após calcular a posição dos pontos transformados, um filtro é aplicado para garantir que apenas os pontos que permanecem dentro dos limites da imagem original sejam utilizados. Os índices dos pontos são, então convertidos para inteiros antes de serem usados para acessar os valores dos pixels na imagem original, afinal de contas não existe um pixel na posição (1.5, 1.5).

Com isso, o resultado é uma imagem que pode ser rotacionada e escalada em tempo real, com a possibilidade de gravação de um vídeo da sessão.

## 2: Explique como funciona a remoção de artefatos e relacione o conceito ao código implementado.

Os artefatos em processamento de imagem, especialmente durante transformações geométricas como rotação e escalonamento, ocorrem devido à discrepância entre a grade discreta de pixels da imagem e as operações contínuas de transformação. Isso pode resultar em "buracos" na imagem de saída, onde alguns pixels não têm valores atribuídos.

Para lidar com esses problemas e remover artefatos, o código implementado utiliza a técnica de transformação inversa. Em vez de mapear cada ponto da imagem de entrada para a imagem de saída (o que pode resultar em buracos ou sobreposições), o código mapeia cada ponto da imagem de saída de volta para a imagem de entrada. Isso garante que todos os pixels na imagem de saída tenham um valor atribuído e reduz a ocorrência de artefatos.

#### Com o código:
1. ##### Transformação Inversa: 

    Em vez de aplicarmos somente a matriz de transformação $X_{final} = T \cdot R \cdot S \cdot T^{-1} \cdot X$ aos pontos da imagem da entrada, o código calcula a inversa dessa matriz e aplica aos pontos da imagem de saída. Isso é feito em:
    
    ```python
    Xd = np.linalg.inv(Xd) @ X
    ```

2. ##### Filtragem dos indíces:
    
    Após aplicarmos a transformação inversa aos pontos da imagem de saída, o código filtra os índices dos pontos para garantir que apenas os pontos que permanecem dentro dos limites da imagem original sejam utilizados.
  
    ```python
    filtro = (Xd[0,:] >= 0) & (Xd[0,:] < image.shape[0]) & (Xd[1,:] >= 0) & (Xd[1,:] < image.shape[1])
    X = X[:,filtro]
    Xd = Xd[:,filtro]
    ```

3. ##### Conversão para inteiros:
        
    Após filtrar os índices dos pontos, o código converte os índices para inteiros antes de serem usados para acessar os valores dos pixels na imagem original.
    ```python
    X = X.astype(int)
    Xd = Xd.astype(int)
    ```
    
4. ##### Atribuição de valores:
    
    Finalmente, os valores dos pixels na imagem de saída são preenchidos com os valores correspondentes na imagem de entrada, usando os índices transformados.
    ```python
    image_[X[0,:], X[1,:]] = image[Xd[0,:], Xd[1,:]]
    ```

Ao mapear os pontos da imagem de saída para a imagem de entrada, o código evita buracos e garante que todos os pixels na imagem de saída tenham um valor atribuído, por sua vez a filtragem de índices remove os pontos que estariam fora dos limites, evitando acessos inválidos à memória e garantindo que os valores dos pixels sejam válidos. Com a junção dessas etapas, podemos minimizar os artefatos decorrentes das transformações geométricas e resultam em uma imagem de saída mais suave e precisa.

## 3: Como foi implementado o processo responsável por salvar o vídeo? Explique o código implementado.

**O processo responsável por salvar o vídeo são:**

O processo de gravação de vídeo foi implementado usando a biblioteca OpenCV no Python.

1. #### Inicialização: 

    No inicio do código:
    - `cv.VideoCapture(0)`: Importamos a biblioteca `OpenCV` e iniciamos esse, uma vez que esse é responsável pela inicialização da captura de vídeo, utilizando a primeira câmera detectada.

    Al'ém disso, definimos as seguintes variáveis:

    ```python
    cap.set(cv.CAP_PROP_FPS, 45)
    recording = False
    fourcc = cv.VideoWriter_fourcc(*'H264')  # Usar o codec H264 para MP4
    out = None
    ```
    
    Onde:
    - `cap.set(cv.CAP_PROP_FPS, 45)`: Define a taxa de quadros da captura de vídeo para 45 quadros por segundo.
    - `recording = False`: Define a variável `recording` como `False`, indicando que a gravação não está ativa.
    - `fourcc = cv.VideoWriter_fourcc(*'H264')`: Define o codec de vídeo como H264, que é compatível com o formato MP4.
    - `out = None`: Define a variável `out` como `None`, indicando que o arquivo de vídeo não foi inicializado.

    <br>

2. #### Iniciar Gravação:

    Ao pressionar a tecla `s`, se `recording` for 'False', ele é definido como 'True' e o 'VideoWriter' é configurado para gravar o vídeo em 'video.mp4' com 20 frames por segundo.

3. #### Gravar Frames: 

    Dentro do loop principal, se recording' estiver 'True', cada frame transformado é escrito no arquivo de vídeo usando `out.write()`.

4. #### Parar Gravação: 

    Ao pressionar a tecla `e`, se "recording" for "True', ele é definido como 'False', o método `release()` é chamado para fechar o arquivo de vídeo, e uma mensagem é impressa indicando que o vídeo foi salvo.

Isso garante que o usuário possa controlar a gravação do vídeo e que os arquivos de vídeo sejam manipulados de maneira segura.